In [6]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import time
import plotly.express as px
import plotly.graph_objects as go
from spot_vol import fetch_binance_btc_spot_price  # From your provided file

def get_friday_dates(start_date, end_date):
    """
    Generate a list of Friday dates at 00:00 UTC within the given date range.
    """
    fridays = []
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 4:  # Friday
            fridays.append(current_date.replace(hour=0, minute=0, second=0, microsecond=0))
        current_date += timedelta(days=1)
    return fridays

def round_strike_price(spot_price, is_call=True):
    """
    Round spot price to nearest 1000 for strike price.
    Calls: ceiling to next 1000 (e.g., 84200 -> 85000)
    Puts: floor to previous 1000 (e.g., 84200 -> 84000)
    """
    if is_call:
        return np.ceil(spot_price / 1000) * 1000
    else:
        return np.floor(spot_price / 1000) * 1000

def generate_deribit_ticker(currency, strike, expiry_date, option_type):
    """
    Generate Deribit ticker in the format: BTC-DMMMYY-STRIKE-C/P
    e.g., BTC-3MAY24-60000-C (no leading zero in day)
    """
    day = str(int(expiry_date.day))  # Remove leading zero
    month_year = expiry_date.strftime("%b%y").upper()
    strike_str = str(int(strike))
    return f"{currency}-{day}{month_year}-{strike_str}-{option_type}"

def fetch_deribit_option_prices(ticker, start_date, end_date, retries=3):
    """
    Fetch hourly option prices for a given Deribit ticker using /public/get_tradingview_chart_data.
    Returns a DataFrame with timestamp and close price.
    """
    url = "https://www.deribit.com/api/v2/public/get_tradingview_chart_data"
    start_ts = int(start_date.timestamp() * 1000)
    end_ts = int(end_date.timestamp() * 1000)
    
    params = {
        "instrument_name": ticker,
        "start_timestamp": start_ts,
        "end_timestamp": end_ts,
        "resolution": "60"  # 60 minutes = 1 hour
    }
    
    for attempt in range(retries):
        try:
            time.sleep(0.1)  # Avoid rate limits
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'result' not in data or 'close' not in data['result']:
                print(f"No price data for {ticker} from {start_date.date()} to {end_date.date()}")
                return pd.DataFrame()
            
            timestamps = [datetime.fromtimestamp(ts / 1000, tz=pytz.UTC) for ts in data['result']['ticks']]
            prices = data['result']['close']
            
            df = pd.DataFrame({
                "timestamp": timestamps,
                "price": prices
            })
            
            df = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]
            df = df.sort_values("timestamp")
            df = df.drop_duplicates(subset=["timestamp"], keep="first")
            
            return df[["timestamp", "price"]]
        
        except requests.RequestException as e:
            print(f"Attempt {attempt+1} failed for {ticker}: {e}")
            if attempt == retries - 1:
                return pd.DataFrame()
            time.sleep(1)

def check_data_availability(ticker, start_date, end_date):
    """
    Check if price data is available for a given ticker and date range.
    """
    df = fetch_deribit_option_prices(ticker, start_date, end_date)
    return not df.empty

def backtest_weekly_straddle(days=365):
    """
    Backtest selling a BTC weekly straddle every Friday at 00:00 UTC.
    Returns a DataFrame with trade results and a DataFrame with hourly PnL.
    """
    end_dt = datetime.now(pytz.UTC).replace(hour=0, minute=0, second=0, microsecond=0)
    start_dt = end_dt - timedelta(days=days)
    
    btc_df = fetch_binance_btc_spot_price(start_dt, end_dt)
    if btc_df.empty:
        print("No BTC spot price data. Cannot proceed with backtest.")
        return pd.DataFrame(), pd.DataFrame()
    
    fridays = get_friday_dates(start_dt, end_dt)
    
    trade_results = []
    hourly_pnl_records = []
    
    for entry_time in fridays:
        entry_date = entry_time.date()
        spot_price_row = btc_df[btc_df["date"] == entry_date]
        
        if spot_price_row.empty:
            print(f"No spot price data for {entry_date}. Skipping trade.")
            continue
        
        spot_price = spot_price_row["price"].iloc[0]
        
        call_strike = round_strike_price(spot_price, is_call=True)
        put_strike = round_strike_price(spot_price, is_call=False)
        
        expiry_time = entry_time + timedelta(days=7)
        expiry_time = expiry_time.replace(hour=8)
        
        call_ticker = generate_deribit_ticker("BTC", call_strike, expiry_time, "C")
        put_ticker = generate_deribit_ticker("BTC", put_strike, expiry_time, "P")
        
        if not (check_data_availability(call_ticker, entry_time, expiry_time) and 
                check_data_availability(put_ticker, entry_time, expiry_time)):
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_prices = fetch_deribit_option_prices(call_ticker, entry_time, expiry_time)
        put_prices = fetch_deribit_option_prices(put_ticker, entry_time, expiry_time)
        
        if call_prices.empty or put_prices.empty:
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_entry = call_prices[call_prices["timestamp"] >= entry_time].iloc[0]
        put_entry = put_prices[put_prices["timestamp"] >= entry_time].iloc[0]
        
        call_exit = call_prices[call_prices["timestamp"] <= expiry_time].iloc[-1]
        put_exit = put_prices[put_prices["timestamp"] <= expiry_time].iloc[-1]
        
        call_pnl = (call_entry["price"] - call_exit["price"]) * 1
        put_pnl = (put_entry["price"] - put_exit["price"]) * 1
        total_pnl = call_pnl + put_pnl
        
        trade_results.append({
            "entry_time": entry_time,
            "call_ticker": call_ticker,
            "put_ticker": put_ticker,
            "call_strike": call_strike,
            "put_strike": put_strike,
            "expiry_time": expiry_time,
            "call_entry_price": call_entry["price"],
            "put_entry_price": put_entry["price"],
            "call_exit_price": call_exit["price"],
            "put_exit_price": put_exit["price"],
            "call_pnl": call_pnl,
            "put_pnl": put_pnl,
            "total_pnl": total_pnl
        })
        
        hourly_prices = pd.merge(
            call_prices[["timestamp", "price"]].rename(columns={"price": "call_price"}),
            put_prices[["timestamp", "price"]].rename(columns={"price": "put_price"}),
            on="timestamp",
            how="outer"
        )
        hourly_prices = hourly_prices[
            (hourly_prices["timestamp"] >= entry_time) &
            (hourly_prices["timestamp"] <= expiry_time)
        ]
        
        hourly_prices["call_price"] = hourly_prices["call_price"].ffill()
        hourly_prices["put_price"] = hourly_prices["put_price"].ffill()
        
        hourly_prices["call_hourly_pnl"] = (call_entry["price"] - hourly_prices["call_price"]) * 1
        hourly_prices["put_hourly_pnl"] = (put_entry["price"] - hourly_prices["put_price"]) * 1
        hourly_prices["total_hourly_pnl"] = hourly_prices["call_hourly_pnl"] + hourly_prices["put_hourly_pnl"]
        hourly_prices["entry_time"] = entry_time
        
        hourly_pnl_records.append(hourly_prices[["timestamp", "entry_time", "call_hourly_pnl", "put_hourly_pnl", "total_hourly_pnl"]])
    
    trade_df = pd.DataFrame(trade_results)
    hourly_pnl_df = pd.concat(hourly_pnl_records, ignore_index=True) if hourly_pnl_records else pd.DataFrame()
    
    if not trade_df.empty:
        trade_df = trade_df.sort_values("entry_time")
    if not hourly_pnl_df.empty:
        hourly_pnl_df = hourly_pnl_df.sort_values(["timestamp", "entry_time"])
    
    return trade_df, hourly_pnl_df

def calculate_max_drawdown(cumulative_pnl):
    """
    Calculate the maximum drawdown in BTC terms from the cumulative PnL series.
    Returns the absolute drawdown value (in BTC).
    """
    if len(cumulative_pnl) == 0:
        return 0.0
    
    running_max = np.maximum.accumulate(cumulative_pnl)
    drawdowns = running_max - cumulative_pnl
    max_drawdown = drawdowns.max() if len(drawdowns) > 0 else 0.0
    return max_drawdown

def calculate_summary_statistics(trade_df):
    """
    Calculate summary statistics for the backtest, including max drawdown.
    Returns a dictionary with stats.
    """
    if trade_df.empty:
        return {
            "number_of_trades": 0,
            "total_pnl": 0.0,
            "average_pnl_per_trade": 0.0,
            "win_rate": 0.0,
            "max_drawdown": 0.0
        }
    
    number_of_trades = len(trade_df)
    total_pnl = trade_df["total_pnl"].sum()
    average_pnl = trade_df["total_pnl"].mean()
    win_rate = (trade_df["total_pnl"] > 0).mean() * 100
    cumulative_pnl = trade_df["total_pnl"].cumsum()
    max_drawdown = calculate_max_drawdown(cumulative_pnl)
    
    return {
        "number_of_trades": number_of_trades,
        "total_pnl": total_pnl,
        "average_pnl_per_trade": average_pnl,
        "win_rate": win_rate,
        "max_drawdown": max_drawdown
    }

def plot_pnl_charts(trade_df):
    """
    Generate two Plotly charts: cumulative PnL curve and PnL per trade bar plot.
    Returns a tuple of (pnl_curve_fig, pnl_bar_fig).
    """
    if trade_df.empty:
        print("No trade data to plot.")
        return None, None
    
    # Cumulative PnL Curve
    trade_df["cumulative_pnl"] = trade_df["total_pnl"].cumsum()
    pnl_curve_fig = px.line(
        trade_df,
        x="entry_time",
        y="cumulative_pnl",
        title="Cumulative PnL Over Time",
        labels={"entry_time": "Date", "cumulative_pnl": "Cumulative PnL (BTC)"},
        color_discrete_sequence=["cyan"]
    )
    pnl_curve_fig.update_layout(
        plot_bgcolor="rgb(30,30,30)",
        paper_bgcolor="rgb(30,30,30)",
        font_color="white",
        xaxis_gridcolor="rgb(80,80,80)",
        yaxis_gridcolor="rgb(80,80,80)",
        showlegend=False,
        width=1000,
        height=600
    )
    
    # PnL per Trade Bar Plot
    trade_df["trade_number"] = range(1, len(trade_df) + 1)
    colors = ["rgb(128, 198, 128)" if x > 0 else "rgb(255, 138, 138)" for x in trade_df["total_pnl"]]
    
    pnl_bar_fig = go.Figure(data=[
        go.Bar(
            x=trade_df["entry_time"],
            y=trade_df["total_pnl"],
            marker_color=colors,
            text=trade_df["total_pnl"].round(2),
            textposition="auto"
        )
    ])
    pnl_bar_fig.update_layout(
        title="PnL per Trade",
        xaxis_title="Trade Entry Date",
        yaxis_title="PnL (BTC)",
        plot_bgcolor="rgb(30,30,30)",
        paper_bgcolor="rgb(30,30,30)",
        font_color="white",
        xaxis_gridcolor="rgb(80,80,80)",
        yaxis_gridcolor="rgb(80,80,80)",
        showlegend=False,
        width=1000,
        height=600,
        xaxis_tickangle=45
    )
    
    return pnl_curve_fig, pnl_bar_fig

def plot_hourly_cumulative_pnl(hourly_pnl_df):
    """
    Generate a Plotly line chart showing the cumulative hourly PnL over the entire backtest period.
    Calculates incremental hourly PnL from total_hourly_pnl, which is cumulative per trade.
    Args:
        hourly_pnl_df (pd.DataFrame): DataFrame with columns ['timestamp', 'entry_time', 'call_hourly_pnl', 'put_hourly_pnl', 'total_hourly_pnl']
    Returns:
        plotly.graph_objects.Figure: Plotly figure object for the cumulative hourly PnL chart
    """
    if hourly_pnl_df.empty:
        print("No hourly PnL data to plot.")
        return None
    
    # Sort by timestamp and entry_time to ensure chronological order
    df = hourly_pnl_df.sort_values(["entry_time", "timestamp"]).copy()
    
    # Calculate incremental hourly PnL for each trade
    df["incremental_pnl"] = df.groupby("entry_time")["total_hourly_pnl"].diff().fillna(df["total_hourly_pnl"])
    
    # Aggregate incremental PnL across all open trades at each timestamp
    aggregated_pnl = df.groupby("timestamp")["incremental_pnl"].sum().reset_index()
    
    # Calculate cumulative PnL over the entire backtest period
    aggregated_pnl["cumulative_pnl"] = aggregated_pnl["incremental_pnl"].cumsum()
    
    # Create line chart
    fig = px.line(
        aggregated_pnl,
        x="timestamp",
        y="cumulative_pnl",
        title="Cumulative Hourly PnL Over Backtest Period",
        labels={"timestamp": "Date and Time (UTC)", "cumulative_pnl": "Cumulative PnL (BTC)"},
        color_discrete_sequence=["cyan"]
    )
    
    # Customize layout
    fig.update_layout(
        plot_bgcolor="rgb(30,30,30)",
        paper_bgcolor="rgb(30,30,30)",
        font_color="white",
        xaxis_gridcolor="rgb(80,80,80)",
        yaxis_gridcolor="rgb(80,80,80)",
        showlegend=False,
        width=1000,
        height=600
    )
    
    return fig

In [18]:

trade_df, hourly_pnl_df = backtest_weekly_straddle(days=800)



In [19]:
# Calculate and display summary statistics
stats = calculate_summary_statistics(trade_df)
print("\nTrade Summary:")
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title()}: {value:.2f}" if isinstance(value, float) else f"{key.replace('_', ' ').title()}: {value}")

# Generate and display plots
pnl_curve_fig, pnl_bar_fig = plot_pnl_charts(trade_df)
if pnl_curve_fig:
    pnl_curve_fig.show()
if pnl_bar_fig:
    pnl_bar_fig.show()


Trade Summary:
Number Of Trades: 114
Total Pnl: 0.30
Average Pnl Per Trade: 0.00
Win Rate: 65.79
Max Drawdown: 0.34


In [20]:
fig = plot_hourly_cumulative_pnl(hourly_pnl_df)
fig.show()

# Only sell if DVOL > 55

In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import time
from spot_vol import fetch_binance_btc_spot_price  # From your provided file

def fetch_dvol_index(trade_time, retries=3):
    """
    Fetch the latest DVOL Index value for BTC before or at the trade time using Deribit API.
    Returns the most recent DVOL value (close price) or None if data is unavailable.
    """
    url = "https://www.deribit.com/api/v2/public/get_volatility_index_data"
    start_ts = int((trade_time - timedelta(hours=24)).timestamp() * 1000)  # 24 hours before
    end_ts = int(trade_time.timestamp() * 1000)
    
    params = {
        "currency": "BTC",
        "start_timestamp": start_ts,
        "end_timestamp": end_ts,
        "resolution": "3600"  # 3600 seconds = 1 hour
    }
    
    for attempt in range(retries):
        try:
            time.sleep(0.1)  # Avoid rate limits
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'result' not in data or 'data' not in data['result']:
                print(f"No DVOL data for BTC from {trade_time - timedelta(hours=24)} to {trade_time}")
                return None
            
            # data['result']['data'] is a list of [timestamp_ms, open, high, low, close]
            dvol_data = data['result']['data']
            if not dvol_data:
                print(f"No DVOL data points available before {trade_time}")
                return None
            
            # Extract timestamps and close values
            df = pd.DataFrame(
                dvol_data,
                columns=["timestamp_ms", "open", "high", "low", "dvol"]
            )
            df["timestamp"] = pd.to_datetime(df["timestamp_ms"], unit="ms", utc=True)
            df = df[df["timestamp"] <= trade_time]
            
            if df.empty:
                print(f"No DVOL data available before {trade_time}")
                return None
            
            # Get the most recent DVOL value
            latest_dvol = df.sort_values("timestamp").iloc[-1]["dvol"]
            return latest_dvol
        
        except requests.RequestException as e:
            print(f"Attempt {attempt+1} failed for DVOL fetch at {trade_time}: {e}")
            if attempt == retries - 1:
                return None
            time.sleep(1)

def backtest_weekly_straddle(days=365, dvol_threshold=55):
    """
    Backtest selling a BTC weekly straddle every Friday at 00:00 UTC if DVOL Index > 55.
    Returns a DataFrame with trade results and a DataFrame with hourly PnL.
    """
    end_dt = datetime.now(pytz.UTC).replace(hour=0, minute=0, second=0, microsecond=0)
    start_dt = end_dt - timedelta(days=days)
    
    btc_df = fetch_binance_btc_spot_price(start_dt, end_dt)
    if btc_df.empty:
        print("No BTC spot price data. Cannot proceed with backtest.")
        return pd.DataFrame(), pd.DataFrame()
    
    fridays = get_friday_dates(start_dt, end_dt)
    
    trade_results = []
    hourly_pnl_records = []
    
    for entry_time in fridays:
        # Check DVOL Index value
        dvol_value = fetch_dvol_index(entry_time)
        if dvol_value is None or dvol_value <= dvol_threshold:
            print(f"Skipping trade at {entry_time}: DVOL={dvol_value if dvol_value else 'N/A'} <= {dvol_threshold}")
            continue
        
        entry_date = entry_time.date()
        spot_price_row = btc_df[btc_df["date"] == entry_date]
        
        if spot_price_row.empty:
            print(f"No spot price data for {entry_date}. Skipping trade.")
            continue
        
        spot_price = spot_price_row["price"].iloc[0]
        
        call_strike = round_strike_price(spot_price, is_call=True)
        put_strike = round_strike_price(spot_price, is_call=False)
        
        expiry_time = entry_time + timedelta(days=7)
        expiry_time = expiry_time.replace(hour=8)
        
        call_ticker = generate_deribit_ticker("BTC", call_strike, expiry_time, "C")
        put_ticker = generate_deribit_ticker("BTC", put_strike, expiry_time, "P")
        
        if not (check_data_availability(call_ticker, entry_time, expiry_time) and 
                check_data_availability(put_ticker, entry_time, expiry_time)):
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_prices = fetch_deribit_option_prices(call_ticker, entry_time, expiry_time)
        put_prices = fetch_deribit_option_prices(put_ticker, entry_time, expiry_time)
        
        if call_prices.empty or put_prices.empty:
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_entry = call_prices[call_prices["timestamp"] >= entry_time].iloc[0]
        put_entry = put_prices[put_prices["timestamp"] >= entry_time].iloc[0]
        
        call_exit = call_prices[call_prices["timestamp"] <= expiry_time].iloc[-1]
        put_exit = put_prices[put_prices["timestamp"] <= expiry_time].iloc[-1]
        
        call_pnl = (call_entry["price"] - call_exit["price"]) * 1
        put_pnl = (put_entry["price"] - put_exit["price"]) * 1
        total_pnl = call_pnl + put_pnl
        
        trade_results.append({
            "entry_time": entry_time,
            "call_ticker": call_ticker,
            "call_strike": call_strike,
            "put_ticker": put_ticker,
            "put_strike": put_strike,
            "expiry_time": expiry_time,
            "call_entry_price": call_entry["price"],
            "put_entry_price": put_entry["price"],
            "call_exit_price": call_exit["price"],
            "put_exit_price": put_exit["price"],
            "call_pnl": call_pnl,
            "put_pnl": put_pnl,
            "total_pnl": total_pnl,
            "dvol_value": dvol_value
        })
        
        hourly_prices = pd.merge(
            call_prices[["timestamp", "price"]].rename(columns={"price": "call_price"}),
            put_prices[["timestamp", "price"]].rename(columns={"price": "put_price"}),
            on="timestamp",
            how="outer"
        )
        hourly_prices = hourly_prices[
            (hourly_prices["timestamp"] >= entry_time) &
            (hourly_prices["timestamp"] <= expiry_time)
        ]
        
        hourly_prices["call_price"] = hourly_prices["call_price"].ffill()
        hourly_prices["put_price"] = hourly_prices["put_price"].ffill()
        
        hourly_prices["call_hourly_pnl"] = (call_entry["price"] - hourly_prices["call_price"]) * 1
        hourly_prices["put_hourly_pnl"] = (put_entry["price"] - hourly_prices["put_price"]) * 1
        hourly_prices["total_hourly_pnl"] = hourly_prices["call_hourly_pnl"] + hourly_prices["put_hourly_pnl"]
        hourly_prices["entry_time"] = entry_time
        
        hourly_pnl_records.append(hourly_prices[["timestamp", "entry_time", "call_hourly_pnl", "put_hourly_pnl", "total_hourly_pnl"]])
    
    trade_df = pd.DataFrame(trade_results)
    hourly_pnl_df = pd.concat(hourly_pnl_records, ignore_index=True) if hourly_pnl_records else pd.DataFrame()
    
    if not trade_df.empty:
        trade_df = trade_df.sort_values("entry_time")
    if not hourly_pnl_df.empty:
        hourly_pnl_df = hourly_pnl_df.sort_values(["timestamp", "entry_time"])
    
    return trade_df, hourly_pnl_df

In [5]:
trade_df, hourly_pnl_df = backtest_weekly_straddle(days=1000, dvol_threshold=15)

NameError: name 'get_friday_dates' is not defined

In [ ]:
# Calculate and display summary statistics
stats = calculate_summary_statistics(trade_df)
print("\nTrade Summary:")
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title()}: {value:.2f}" if isinstance(value, float) else f"{key.replace('_', ' ').title()}: {value}")

# Generate and display plots
pnl_curve_fig, pnl_bar_fig = plot_pnl_charts(trade_df)
if pnl_curve_fig:
    pnl_curve_fig.show()
if pnl_bar_fig:
    pnl_bar_fig.show()


Trade Summary:
Number Of Trades: 52
Total Pnl: 0.64
Average Pnl Per Trade: 0.01
Win Rate: 69.23
Max Drawdown: 0.13


In [31]:
fig = plot_hourly_cumulative_pnl(hourly_pnl_df)
fig.show()

In [39]:
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
from spot_vol import fetch_binance_btc_spot_price, clean_ohlc_data

def calculate_btc_realized_volatility(start_date: str, end_date: str, lookback_window: int) -> dict:
    """
    Calculate annualized realized volatility for BTC using daily and hourly close prices from Binance API.
    
    Parameters:
    - start_date (str): Start date in 'YYYY-MM-DD' format (e.g., '2024-01-01').
    - end_date (str): End date in 'YYYY-MM-DD' format (e.g., '2024-12-31').
    - lookback_window (int): Number of periods for volatility calculation (e.g., 30).
    
    Returns:
    - dict: Dictionary containing 'daily_rv' and 'hourly_rv' as pandas Series with annualized volatility.
    """
    # Convert input dates to datetime objects with UTC timezone
    try:
        start_dt = pd.to_datetime(start_date).replace(tzinfo=pytz.UTC)
        end_dt = pd.to_datetime(end_date).replace(tzinfo=pytz.UTC)
    except ValueError as e:
        raise ValueError("Invalid date format. Use 'YYYY-MM-DD'.") from e

    if start_dt >= end_dt:
        raise ValueError("Start date must be before end date.")
    if lookback_window <= 0:
        raise ValueError("Lookback window must be positive.")

    # Fetch hourly BTC spot price data
    df_hourly = fetch_binance_btc_spot_price(start_dt, end_dt)
    if df_hourly.empty:
        raise ValueError("No hourly data retrieved from Binance API.")

    # Extract daily close prices (already filtered for 00:00 UTC in fetch function)
    df_daily = df_hourly.copy()
    df_daily['timestamp'] = pd.to_datetime(df_daily['date'])
    df_daily = df_daily.set_index('timestamp')[['price']].rename(columns={'price': 'close'})

    # Fetch hourly data again without filtering for 00:00 UTC to get full hourly series
    url = "https://api.binance.us/api/v3/klines"
    chunk_days = 30
    all_data = []

    current_start = start_dt
    while current_start < end_dt:
        current_end = min(current_start + pd.Timedelta(days=chunk_days), end_dt)
        start_ts = int(current_start.timestamp() * 1000)
        end_ts = int(current_end.timestamp() * 1000)

        params = {
            "symbol": "BTCUSDT",
            "interval": "1h",
            "startTime": start_ts,
            "endTime": end_ts,
            "limit": 1000
        }

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            if not data:
                current_start = current_end
                continue

            df = pd.DataFrame(
                data,
                columns=[
                    "timestamp", "open", "high", "low", "close", "volume",
                    "close_time", "quote_asset_volume", "number_of_trades",
                    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
                ]
            )

            df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
            df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
            df = clean_ohlc_data(df)
            all_data.append(df)

        except requests.RequestException as e:
            print(f"Error fetching hourly data for {current_start.date()} to {current_end.date()}: {e}")

        current_start = current_end

    if not all_data:
        raise ValueError("No hourly data retrieved from Binance API.")

    df_hourly_full = pd.concat(all_data, ignore_index=True)
    df_hourly_full = df_hourly_full.drop_duplicates(subset=["timestamp"], keep="first")
    df_hourly_full = df_hourly_full.sort_values("timestamp")
    df_hourly_full = df_hourly_full.set_index('timestamp')[['close']]

    # Helper function to compute annualized RV
    def compute_rv(prices: pd.Series, window: int, periods_per_year: float) -> pd.Series:
        log_returns = np.log(prices / prices.shift(1))
        rv = log_returns.rolling(window=window).std() * np.sqrt(periods_per_year)
        return rv

    # Annualization factors
    DAYS_PER_YEAR = 365
    HOURS_PER_YEAR = 365 * 24

    # Compute realized volatility
    daily_rv = compute_rv(df_daily['close'], lookback_window, DAYS_PER_YEAR)
    hourly_rv = compute_rv(df_hourly_full['close'], lookback_window, HOURS_PER_YEAR)

    return {
        'daily_rv': daily_rv,
        'hourly_rv': hourly_rv
    }


In [ ]:
start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = input("Enter end date (YYYY-MM-DD): ")
lookback_window = int(input("Enter lookback window (e.g., 30): "))

try:
    rv_results = calculate_btc_realized_volatility(start_date, end_date, lookback_window)
    
    # Print latest values
    print("Latest Daily RV:", rv_results['daily_rv'].dropna().tail(1).values[0])
    print("Latest Hourly RV:", rv_results['hourly_rv'].dropna().tail(1).values[0])

    # Optional: Plot for comparison
    import plotly.graph_objects as go
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=rv_results['daily_rv'].index,
        y=rv_results['daily_rv'],
        mode='lines',
        name='Daily RV',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=rv_results['hourly_rv'].index,
        y=rv_results['hourly_rv'],
        mode='lines',
        name='Hourly RV',
        line=dict(color='orange')
    ))
    fig.update_layout(
        title='BTC Annualized Realized Volatility Comparison',
        xaxis_title='Date',
        yaxis_title='Annualized Volatility',
        plot_bgcolor='white',
        showlegend=True
    )
    fig.show()

except ValueError as e:
    print(f"Error: {e}")

In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
import vrp  # Assuming the provided vrp.py is available as a module

# Function to calculate R² score
def calculate_r2(y_true, y_pred):
    """Calculate R² score for regression fit."""
    ss_total = np.sum((y_true - np.mean(y_true))**2)
    ss_residual = np.sum((y_true - y_pred)**2)
    return 1 - ss_residual / ss_total if ss_total != 0 else 0

# Function to create scatter plot of implied VRP vs realized VRP
def create_vrp_relationship_scatter(start_date, end_date, asset="BTC"):
    if asset not in ["BTC", "ETH"]:
        raise ValueError("Asset must be 'BTC' or 'ETH'")
    
    # Map asset to currency and symbol
    asset_info = {
        "BTC": {"currency": "BTC", "symbol": "BTCUSDT", "name": "Bitcoin"},
        "ETH": {"currency": "ETH", "symbol": "ETHUSDT", "name": "Ethereum"}
    }[asset]

    # Extend end date for realized VRP calculation
    extended_end_date = end_date + timedelta(days=30)
    start_timestamp = int(start_date.timestamp() * 1000)
    end_timestamp = int(extended_end_date.timestamp() * 1000)

    # Fetch DVOL data (IV)
    dvol_df = vrp.get_dvol_data(asset_info["currency"], start_timestamp, end_timestamp, "86400")
    dvol_df.set_index("timestamp", inplace=True)

    # Fetch Binance data
    binance_df = vrp.get_binance_data(asset_info["symbol"], start_date, extended_end_date)
    binance_df.set_index("timestamp", inplace=True)

    # Calculate Garman-Klass volatility
    binance_df["gk_vol"] = vrp.garman_klass_volatility(binance_df, window=30, trading_periods=365)

    # Calculate shifted RV for realized VRP
    shifted_rv = binance_df[["gk_vol"]].shift(-30, freq="D")

    # Merge dataframes for implied VLP
    implied_df = pd.merge(
        dvol_df,
        binance_df[["gk_vol"]],
        left_index=True,
        right_index=True,
        how="inner"
    )
    implied_df["implied_vrp"] = implied_df["dvol"] - implied_df["gk_vol"]

    # Merge dataframes for realized VRP
    realized_df = pd.merge(
        dvol_df,
        shifted_rv.rename(columns={"gk_vol": "shifted_gk_vol"}),
        left_index=True,
        right_index=True,
        how="inner"
    )
    realized_df["realized_vrp"] = realized_df["dvol"] - realized_df["shifted_gk_vol"]

    # Combine implied and realized VRPs
    merged_df = pd.merge(
        implied_df[["implied_vrp"]],
        realized_df[["realized_vrp"]],
        left_index=True,
        right_index=True,
        how="inner"
    )

    # Trim to original date range
    merged_df = merged_df.loc[start_date:end_date]

    # Extract year for coloring
    merged_df["year"] = merged_df.index.year

    # Create scatter plot
    fig = go.Figure()

    # Define colors for each year
    colors = {
        2022: "rgb(31, 119, 180)",    # Blue
        2023: "rgb(255, 127, 14)",    # Orange
        2024: "rgb(44, 160, 44)",     # Green
        2025: "rgb(214, 39, 40)"      # Red
    }

    # Add scatter traces for each year
    for year in sorted(merged_df["year"].unique()):
        year_data = merged_df[merged_df["year"] == year]
        fig.add_trace(
            go.Scatter(
                x=year_data["implied_vrp"] * 100,  # Convert to percentage
                y=year_data["realized_vrp"] * 100,  # Convert to percentage
                mode="markers",
                name=str(year),
                marker=dict(
                    size=8,
                    color=colors.get(year, "rgb(128, 128, 128)"),
                    opacity=0.6
                ),
                text=year_data.index.strftime("%Y-%m-%d"),
                hovertemplate="Date: %{text}<br>Implied VRP: %{x:.1f}%<br>Realized VRP: %{y:.1f}%<extra></extra>"
            )
        )

    # Calculate trend line using linear regression
    x = merged_df["implied_vrp"] * 100  # Implied VRP in percentage
    y = merged_df["realized_vrp"] * 100  # Realized VRP in percentage
    valid = ~(x.isna() | y.isna())
    x_valid = x[valid]
    y_valid = y[valid]
    if len(x_valid) > 1:
        coeffs = np.polyfit(x_valid, y_valid, 1)
        x_trend = np.linspace(x_valid.min(), x_valid.max(), 100)
        y_trend = np.polyval(coeffs, x_trend)
        r2 = calculate_r2(y_valid, np.polyval(coeffs, x_valid))
        print(f"{asset_info['name']} Implied vs Realized VRP Linear Trend R²: {r2:.4f}")

        # Add trend line trace
        fig.add_trace(
            go.Scatter(
                x=x_trend,
                y=y_trend,
                mode="lines",
                name="Linear Trend",
                line=dict(color="rgb(71, 217, 174)", dash="dash"),
                hoverinfo="skip"
            )
        )

    # Update layout
    fig.update_layout(
        title=f"{asset_info['name']} Implied VRP vs Realized VRP",
        xaxis_title="Implied VRP (IV - RV, %)",
        yaxis_title="Realized VRP (IV - Future RV, %)",
        template="plotly_white",
        hovermode="closest",
        showlegend=True,
        height=600,
        width=800
    )

    # Update axes
    fig.update_xaxes(tickformat=".1f", title_standoff=10)
    fig.update_yaxes(tickformat=".1f", title_standoff=10)

    return fig



In [7]:

start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 4, 24)
fig = create_vrp_relationship_scatter(start_date, end_date, asset="BTC")
fig.show()

Bitcoin Implied vs Realized VRP Linear Trend R²: 0.0001


In [8]:
import pandas as pd
import numpy as np

def backtest_btc_dvol_strategy(file_path):
    # Load the data
    df = pd.read_csv(file_path)
    df['time'] = pd.to_datetime(df['time'], format='%d/%m/%Y')
    df = df.set_index('time')
    
    # Calculate 200-day 1st percentile for DVOL
    df['dvol_200d_p1'] = df['dvol'].rolling(window=200).quantile(0.01)
    
    # Identify days when DVOL dips below its 200-day 1st percentile
    df['signal'] = df['dvol'] < df['dvol_200d_p1']
    
    # Calculate BTC forward returns for 1, 3, 5, and 10 days
    df['btc_ret_1d'] = df['btc'].pct_change(1).shift(-1)
    df['btc_ret_3d'] = df['btc'].pct_change(3).shift(-3)
    df['btc_ret_5d'] = df['btc'].pct_change(5).shift(-5)
    df['btc_ret_10d'] = df['btc'].pct_change(10).shift(-10)
    
    # Filter for signal days and select relevant columns
    result = df[df['signal']][['dvol', 'btc', 'btc_ret_1d', 'btc_ret_3d', 'btc_ret_5d', 'btc_ret_10d']]
    
    # Reset index to include date in the output
    result = result.reset_index()
    
    return result

In [9]:
result_df = backtest_btc_dvol_strategy('btc_dvol.csv')
result_df

,time,dvol,btc,btc_ret_1d,btc_ret_3d,btc_ret_5d,btc_ret_10d
0,2021-12-25,81.08,50428.31,0.007406,-0.057214,-0.065563,-0.091490
1,2021-12-26,80.26,50801.79,-0.001654,-0.085244,-0.090362,-0.144990
2,2021-12-28,76.70,47543.09,-0.022545,-0.028014,-0.005133,-0.125737
3,2021-12-31,75.51,46211.24,0.032940,0.005374,-0.060055,-0.094937
4,2022-01-01,70.91,47733.43,-0.009100,-0.040199,-0.097409,-0.104329
...,...,...,...,...,...,...,...
58,2025-03-24,47.32,87523.62,-0.001094,-0.003498,-0.056062,-0.049693
59,2025-03-25,46.99,87427.88,-0.005740,-0.034841,-0.057738,-0.040808
60,2025-03-27,46.61,87217.48,-0.032513,-0.055465,-0.023471,-0.101433
61,2025-04-18,46.90,84466.47,0.007204,0.036175,0.109800,0.122018


In [21]:
def backtest_btc_dvol_strategy_extended(file_path):
    # Load the data
    df = pd.read_csv(file_path)
    df['time'] = pd.to_datetime(df['time'], format='%d/%m/%Y')
    df = df.set_index('time')
    
    # Calculate 200-day 1st percentile for DVOL
    df['dvol_200d_p1'] = df['dvol'].rolling(window=200).quantile(0.01)
    
    # Identify days when DVOL dips below its 200-day 1st percentile
    df['signal'] = df['dvol'] < 47
    
    # Calculate BTC forward returns for 1, 3, 5, and 10 days
    periods = [1, 3, 5, 10]
    for p in periods:
        df[f'btc_ret_{p}d'] = df['btc'].pct_change(p).shift(-p)
        # Absolute returns
        df[f'btc_abs_ret_{p}d'] = df[f'btc_ret_{p}d'].abs()
        # DVOL-implied SD: DVOL * sqrt(days/365)
        df[f'implied_sd_{p}d'] = df['dvol'] * np.sqrt(p / 365) / 100  # DVOL is in percentage
        # SD moves: actual return / implied SD
        df[f'sd_move_{p}d'] = abs(df[f'btc_ret_{p}d']) / df[f'implied_sd_{p}d']
    
    # Filter for signal days and select relevant columns
    columns = ['dvol', 'btc'] + [f'btc_ret_{p}d' for p in periods] + \
              [f'btc_abs_ret_{p}d' for p in periods] + \
              [f'implied_sd_{p}d' for p in periods] + \
              [f'sd_move_{p}d' for p in periods]
    result = df[df['signal']][columns]
    
    # Reset index to include date in the output
    result = result.reset_index()
    
    return result

In [22]:
result_df = backtest_btc_dvol_strategy_extended('btc_dvol.csv')
result_df

,time,dvol,btc,btc_ret_1d,btc_ret_3d,btc_ret_5d,btc_ret_10d,btc_abs_ret_1d,btc_abs_ret_3d,btc_abs_ret_5d,btc_abs_ret_10d,implied_sd_1d,implied_sd_3d,implied_sd_5d,implied_sd_10d,sd_move_1d,sd_move_3d,sd_move_5d,sd_move_10d
0,2023-01-01,46.96,16611.58,0.003333,0.014017,0.020256,0.080227,0.003333,0.014017,0.020256,0.080227,0.024580,0.042574,0.054963,0.077729,0.135607,0.329234,0.368537,1.032135
1,2023-01-04,46.87,16844.42,-0.001112,0.005815,0.019802,0.244152,0.001112,0.005815,0.019802,0.244152,0.024533,0.042492,0.054857,0.077580,0.045325,0.136848,0.360981,3.147110
2,2023-01-05,45.46,16825.69,0.007273,0.017800,0.036718,0.240942,0.007273,0.017800,0.036718,0.240942,0.023795,0.041214,0.053207,0.075246,0.305646,0.431883,0.690103,3.202056
3,2023-01-06,43.61,16948.06,-0.000336,0.013566,0.058780,0.250143,0.000336,0.013566,0.058780,0.250143,0.022827,0.039537,0.051042,0.072184,0.014708,0.343128,1.151612,3.465364
4,2023-01-07,42.01,16942.37,0.010790,0.029579,0.112462,0.247543,0.010790,0.029579,0.112462,0.247543,0.021989,0.038086,0.049169,0.069535,0.490704,0.776621,2.287263,3.559950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,2025-03-25,46.99,87427.88,-0.005740,-0.034841,-0.057738,-0.040808,0.005740,0.034841,0.057738,0.040808,0.024596,0.042601,0.054998,0.077778,0.233390,0.817846,1.049825,0.524665
169,2025-03-27,46.61,87217.48,-0.032513,-0.055465,-0.023471,-0.101433,0.032513,0.055465,0.023471,0.101433,0.024397,0.042256,0.054553,0.077149,1.332665,1.312575,0.430249,1.314760
170,2025-04-18,46.90,84466.47,0.007204,0.036175,0.109800,0.122018,0.007204,0.036175,0.109800,0.122018,0.024549,0.042519,0.054892,0.077629,0.293470,0.850795,2.000287,1.571801
171,2025-04-24,46.96,94021.96,0.007305,-0.002567,NaN,NaN,0.007305,0.002567,NaN,NaN,0.024580,0.042574,0.054963,0.077729,0.297193,0.060304,NaN,NaN


In [26]:
fig = px.scatter(result_df, x = 'dvol', y = 'z_score_10d')
fig.show()

In [23]:
result_df.to_csv('results.csv')

In [33]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def calculate_zscores_full_history(file_path):
    # Load the data
    df = pd.read_csv(file_path)
    df['time'] = pd.to_datetime(df['time'], format='%d/%m/%Y')
    df = df.set_index('time')
    
    # Calculate BTC forward returns and z-scores for 1, 3, 5, and 10 days
    periods = [1, 3, 5, 10]
    for p in periods:
        # BTC returns
        df[f'btc_ret_{p}d'] = df['btc'].pct_change(p).shift(-p)
        # DVOL-implied SD: DVOL * sqrt(days/365)
        df[f'implied_sd_{p}d'] = df['dvol'] * np.sqrt(p / 365) / 100  # DVOL is in percentage
        # Z-scores: actual return / implied SD
        df[f'z_score_{p}d'] = abs(df[f'btc_ret_{p}d']) / df[f'implied_sd_{p}d']
    
    # Select relevant columns
    columns = ['dvol', 'btc'] + [f'z_score_{p}d' for p in periods]
    result = df[columns].reset_index()
    
    return result

def plot_10day_zscore_scatter(result_df):
    # Prepare data
    z_score_col = 'z_score_10d'
    dvol = result_df['dvol']
    z_scores = result_df[z_score_col]
    
    # Create scatter plot
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=dvol,
            y=z_scores,
            mode='markers',
            marker=dict(size=8, color='blue', opacity=0.6),
            name='10-Day Z-Score'
        )
    )
    
    # Update layout
    fig.update_layout(
        title='10D Return vs IV',
        xaxis_title='IV',
        yaxis_title='10D Return Z-Score',
        height=500,
        width=800,
        showlegend=True
    )
    
    # Show plot
    fig.show()

# Example usage:
# result_df = calculate_zscores_full_history('btc_dvol.csv')
# plot_10day_zscore_scatter(result_df)
result_df = calculate_zscores_full_history('btc_dvol.csv')
plot_10day_zscore_scatter(result_df)

In [35]:
import pandas as pd
import yfinance as yf
import numpy as np
import plotly.graph_objects as go

def calculate_spy_zscore_vix(start_date='2020-01-01', end_date='2025-04-28'):
    # Fetch SPY and VIX data from yfinance
    spy = yf.download('SPY', start=start_date, end=end_date, progress=False)
    vix = yf.download('^VIX', start=start_date, end=end_date, progress=False)
    
    # Check if data is empty
    if spy.empty or vix.empty:
        raise ValueError("Failed to fetch data for SPY or VIX. Check ticker symbols or date range.")
    
    # Align the data by joining on common dates
    df = pd.DataFrame(index=spy.index)
    df['spy'] = spy['Close']
    df['vix'] = vix['Close']
    
    # Drop rows with missing values
    df = df.dropna()
    
    # Check if DataFrame has enough data
    if len(df) < 10:
        raise ValueError("Insufficient data after alignment. Need at least 10 days of data.")
    
    # Calculate 10-day forward returns for SPY
    df['spy_ret_10d'] = df['spy'].pct_change(10).shift(-10)
    
    # Calculate VIX-implied SD for 10 days: VIX * sqrt(10/365)
    df['implied_sd_10d'] = df['vix'] * np.sqrt(10 / 365) / 100  # VIX is in percentage
    
    # Calculate z-score: actual return / implied SD
    df['z_score_10d'] = abs(df['spy_ret_10d']) / df['implied_sd_10d']
    
    # Drop NaN values and reset index
    result = df[['vix', 'spy', 'z_score_10d']].dropna().reset_index()
    result.rename(columns={'Date': 'time'}, inplace=True)
    
    return result

def plot_spy_10day_zscore_scatter(result_df):
    # Prepare data
    z_score_col = 'z_score_10d'
    vix = result_df['vix']
    z_scores = result_df[z_score_col]
    
    # Create scatter plot
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x=vix,
            y=z_scores,
            mode='markers',
            marker=dict(size=8, color='blue', opacity=0.6),
            name='10-Day Z-Score'
        )
    )
    
    # Update layout
    fig.update_layout(
        title='SPY 10-Day Z-Score vs VIX',
        xaxis_title='VIX',
        yaxis_title='10-Day Z-Score',
        height=500,
        width=800,
        showlegend=True
    )
    
    # Show plot
    fig.show()


result_df = calculate_spy_zscore_vix()
plot_spy_10day_zscore_scatter(result_df)

In [75]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import math

# Function to clean OHLC data by dropping rows with outliers
def clean_ohlc_data(df):
    """
    Cleans OHLC data by dropping rows where any OHLC value is over 2x or under 0.5x
    the median of adjacent points (previous and next day).
    """
    df_clean = df.copy()
    columns_to_check = ["open", "high", "low", "close"]
    
    # Initialize a mask for rows to keep (True means keep, False means drop)
    keep_rows = pd.Series(True, index=df_clean.index)
    
    for col in columns_to_check:
        # Calculate median of previous and next day for each point
        prev_values = df_clean[col].shift(1)
        next_values = df_clean[col].shift(-1)
        median_adjacent = pd.concat([prev_values, next_values], axis=1).median(axis=1)
        
        # Identify outliers: >2x or <0.5x the median of adjacent points
        ratio = df_clean[col] / median_adjacent
        outliers = (ratio > 2) | (ratio < 0.5)
        
        # Update mask: mark rows with outliers to be dropped
        keep_rows = keep_rows & ~outliers
    
    # Drop rows where any column has an outlier
    df_clean = df_clean[keep_rows]
    
    return df_clean

# Function to fetch Binance OHLC data using public API
def get_binance_data(symbol, start_date, end_date):
    # Convert date objects to datetime if necessary
    if isinstance(start_date, date) and not isinstance(start_date, datetime):
        start_date = datetime.combine(start_date, datetime.min.time())
    if isinstance(end_date, date) and not isinstance(end_date, datetime):
        end_date = datetime.combine(end_date, datetime.min.time())
    
    url = "https://api.binance.us/api/v3/klines"
    start_ts = int(start_date.timestamp() * 1000)
    end_ts = int(end_date.timestamp() * 1000)
    params = {
        "symbol": symbol,
        "interval": "1d",
        "startTime": start_ts,
        "endTime": end_ts,
        "limit": 1000
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(
            data,
            columns=[
                "timestamp",
                "open",
                "high",
                "low",
                "close",
                "volume",
                "close_time",
                "quote_asset_volume",
                "number_of_trades",
                "taker_buy_base_asset_volume",
                "taker_buy_quote_asset_volume",
                "ignore",
            ],
        )
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
        df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
        
        # Clean the OHLC data
        df = clean_ohlc_data(df)
        
        return df[["timestamp", "open", "high", "low", "close"]]
    else:
        raise Exception(f"Binance API error: {response.status_code}")

# Function to calculate Garman-Klass volatility
def garman_klass_volatility(price_data, window=14, trading_periods=365, clean=True):
    log_hl = (price_data["high"] / price_data["low"]).apply(np.log)
    log_co = (price_data["close"] / price_data["open"]).apply(np.log)
    rs = 0.5 * log_hl**2 - (2 * math.log(2) - 1) * log_co**2
    def f(v):
        return (trading_periods * v.mean()) ** 0.5
    result = rs.rolling(window=window, center=False).apply(func=f)
    if clean:
        return result.dropna()
    return result

# Function to get DataFrame of low RV dates and returns
def get_low_rv_returns_df(start_date, end_date, symbol="BTCUSDT"):
    # Convert date objects to datetime if necessary
    if isinstance(start_date, date) and not isinstance(start_date, datetime):
        start_date = datetime.combine(start_date, datetime.min.time())
    if isinstance(end_date, date) and not isinstance(end_date, datetime):
        end_date = datetime.combine(end_date, datetime.min.time())
    
    # Fetch Binance data
    # Extend end_date by 14 days to calculate future returns
    extended_end_date = end_date + timedelta(days=14)
    df = get_binance_data(symbol, start_date, extended_end_date)
    df.set_index("timestamp", inplace=True)
    
    # Calculate 14-day RV
    df["rv_14d"] = garman_klass_volatility(df, window=14, trading_periods=365)
    
    # Identify days when 14-day RV is below 3rd percentile
    rv_14d = df["rv_14d"]
    percentile = np.percentile(rv_14d.dropna(), 15)
    low_rv_days = rv_14d < 0.4
    
    # Calculate future returns for 1 to 14 days
    for days in range(1, 15):
        df[f"return_{days}d"] = df["close"].pct_change(periods=days).shift(-days) * 100  # In percentage
    
    # Create DataFrame for low RV days
    result_df = df[low_rv_days][["close", "rv_14d"]].copy()
    result_df = result_df.rename(columns={"close": "BTC_Price", "rv_14d": "14D_RV (%)"})
    
    # Convert RV to percentage
    result_df["14D_RV (%)"] = result_df["14D_RV (%)"] * 100
    
    # Add returns columns
    for days in range(1, 15):
        result_df[f"Return_{days}D (%)"] = df[f"return_{days}d"][low_rv_days]
    
    # Reset index to make date a column
    result_df = result_df.reset_index()
    result_df = result_df.rename(columns={"timestamp": "Date"})
    
    # Sort by date and round numeric columns
    result_df = result_df.sort_values("Date")
    numeric_columns = ["BTC_Price", "14D_RV (%)"] + [f"Return_{days}D (%)" for days in range(1, 15)]
    result_df[numeric_columns] = result_df[numeric_columns].round(2)
    
    return result_df


In [76]:
# Define date range
end_date = date.today()
start_date = end_date - timedelta(days=365*3)  # Last 3 years of data

# Get DataFrame
low_rv_df = get_low_rv_returns_df(start_date, end_date)

In [77]:
low_rv_df.to_csv('results.csv')

In [78]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import time
from spot_vol import fetch_binance_btc_spot_price  # From your provided file

def fetch_dvol_index(trade_time, retries=3):
    """
    Fetch the latest DVOL Index value for BTC before or at the trade time using Deribit API.
    Returns the most recent DVOL value (close price) or None if data is unavailable.
    """
    url = "https://www.deribit.com/api/v2/public/get_volatility_index_data"
    start_ts = int((trade_time - timedelta(hours=24)).timestamp() * 1000)  # 24 hours before
    end_ts = int(trade_time.timestamp() * 1000)
    
    params = {
        "currency": "BTC",
        "start_timestamp": start_ts,
        "end_timestamp": end_ts,
        "resolution": "3600"  # 3600 seconds = 1 hour
    }
    
    for attempt in range(retries):
        try:
            time.sleep(0.1)  # Avoid rate limits
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'result' not in data or 'data' not in data['result']:
                print(f"No DVOL data for BTC from {trade_time - timedelta(hours=24)} to {trade_time}")
                return None
            
            # data['result']['data'] is a list of [timestamp_ms, open, high, low, close]
            dvol_data = data['result']['data']
            if not dvol_data:
                print(f"No DVOL data points available before {trade_time}")
                return None
            
            # Extract timestamps and close values
            df = pd.DataFrame(
                dvol_data,
                columns=["timestamp_ms", "open", "high", "low", "dvol"]
            )
            df["timestamp"] = pd.to_datetime(df["timestamp_ms"], unit="ms", utc=True)
            df = df[df["timestamp"] <= trade_time]
            
            if df.empty:
                print(f"No DVOL data available before {trade_time}")
                return None
            
            # Get the most recent DVOL value
            latest_dvol = df.sort_values("timestamp").iloc[-1]["dvol"]
            return latest_dvol
        
        except requests.RequestException as e:
            print(f"Attempt {attempt+1} failed for DVOL fetch at {trade_time}: {e}")
            if attempt == retries - 1:
                return None
            time.sleep(1)

def fetch_dvol_for_period(start_time, end_time, retries=3):
    """
    Fetch DVOL Index values for BTC over a period using Deribit API.
    Returns a DataFrame with timestamp and dvol columns or empty DataFrame if unavailable.
    """
    url = "https://www.deribit.com/api/v2/public/get_volatility_index_data"
    start_ts = int(start_time.timestamp() * 1000)
    end_ts = int(end_time.timestamp() * 1000)
    
    params = {
        "currency": "BTC",
        "start_timestamp": start_ts,
        "end_timestamp": end_ts,
        "resolution": "3600"  # 3600 seconds = 1 hour
    }
    
    for attempt in range(retries):
        try:
            time.sleep(0.1)  # Avoid rate limits
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'result' not in data or 'data' not in data['result']:
                print(f"No DVOL data for BTC from {start_time} to {end_time}")
                return pd.DataFrame()
            
            dvol_data = data['result']['data']
            if not dvol_data:
                print(f"No DVOL data points available from {start_time} to {end_time}")
                return pd.DataFrame()
            
            df = pd.DataFrame(
                dvol_data,
                columns=["timestamp_ms", "open", "high", "low", "dvol"]
            )
            df["timestamp"] = pd.to_datetime(df["timestamp_ms"], unit="ms", utc=True)
            df = df[["timestamp", "dvol"]]
            df = df[(df["timestamp"] >= start_time) & (df["timestamp"] <= end_time)]
            df = df.sort_values("timestamp")
            
            return df
        
        except requests.RequestException as e:
            print(f"Attempt {attempt+1} failed for DVOL fetch from {start_time} to {end_time}: {e}")
            if attempt == retries - 1:
                return pd.DataFrame()
            time.sleep(1)

def backtest_weekly_straddle_buy_low_dvol(days=365, dvol_buy_threshold=45, dvol_sell_threshold=55):
    """
    Backtest buying a BTC weekly straddle every Friday at 00:00 UTC if DVOL < 45,
    selling when DVOL > 55 or at expiry. Returns trade results and hourly PnL DataFrames.
    """
    end_dt = datetime.now(pytz.UTC).replace(hour=0, minute=0, second=0, microsecond=0)
    start_dt = end_dt - timedelta(days=days)
    
    btc_df = fetch_binance_btc_spot_price(start_dt, end_dt)
    if btc_df.empty:
        print("No BTC spot price data. Cannot proceed with backtest.")
        return pd.DataFrame(), pd.DataFrame()
    
    fridays = get_friday_dates(start_dt, end_dt)
    
    trade_results = []
    hourly_pnl_records = []
    
    for entry_time in fridays:
        # Check DVOL Index value for entry
        dvol_value = fetch_dvol_index(entry_time)
        if dvol_value is None or dvol_value >= dvol_buy_threshold:
            print(f"Skipping trade at {entry_time}: DVOL={dvol_value if dvol_value else 'N/A'} >= {dvol_buy_threshold}")
            continue
        
        entry_date = entry_time.date()
        spot_price_row = btc_df[btc_df["date"] == entry_date]
        
        if spot_price_row.empty:
            print(f"No spot price data for {entry_date}. Skipping trade.")
            continue
        
        spot_price = spot_price_row["price"].iloc[0]
        
        call_strike = round_strike_price(spot_price, is_call=True)
        put_strike = round_strike_price(spot_price, is_call=False)
        
        expiry_time = entry_time + timedelta(days=7)
        expiry_time = expiry_time.replace(hour=8)
        
        call_ticker = generate_deribit_ticker("BTC", call_strike, expiry_time, "C")
        put_ticker = generate_deribit_ticker("BTC", put_strike, expiry_time, "P")
        
        if not (check_data_availability(call_ticker, entry_time, expiry_time) and 
                check_data_availability(put_ticker, entry_time, expiry_time)):
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_prices = fetch_deribit_option_prices(call_ticker, entry_time, expiry_time)
        put_prices = fetch_deribit_option_prices(put_ticker, entry_time, expiry_time)
        
        if call_prices.empty or put_prices.empty:
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        # Get entry prices
        call_entry = call_prices[call_prices["timestamp"] >= entry_time].iloc[0]
        put_entry = put_prices[put_prices["timestamp"] >= entry_time].iloc[0]
        
        # Fetch DVOL for the trade period to check exit condition
        dvol_df = fetch_dvol_for_period(entry_time, expiry_time)
        if dvol_df.empty:
            print(f"No DVOL data for trade period {entry_time} to {expiry_time}. Skipping trade.")
            continue
        
        # Find exit time: first timestamp where DVOL > 55, or expiry
        exit_candidates = dvol_df[dvol_df["dvol"] > dvol_sell_threshold]
        if not exit_candidates.empty:
            exit_time = exit_candidates["timestamp"].iloc[0]
        else:
            exit_time = expiry_time
        
        # Get exit prices
        call_exit = call_prices[call_prices["timestamp"] <= exit_time].iloc[-1]
        put_exit = put_prices[put_prices["timestamp"] <= exit_time].iloc[-1]
        
        # Calculate trade PnL (buy at entry, sell at exit)
        call_pnl = (call_exit["price"] - call_entry["price"]) * 1  # Buy: -entry + exit
        put_pnl = (put_exit["price"] - put_entry["price"]) * 1    # Buy: -entry + exit
        total_pnl = call_pnl + put_pnl
        
        trade_results.append({
            "entry_time": entry_time,
            "call_ticker": call_ticker,
            "call_strike": call_strike,
            "put_ticker": put_ticker,
            "put_strike": put_strike,
            "exit_time": exit_time,
            "call_entry_price": call_entry["price"],
            "put_entry_price": put_entry["price"],
            "call_exit_price": call_exit["price"],
            "put_exit_price": put_exit["price"],
            "call_pnl": call_pnl,
            "put_pnl": put_pnl,
            "total_pnl": total_pnl,
            "dvol_entry": dvol_value,
            "dvol_exit": dvol_df[dvol_df["timestamp"] <= exit_time]["dvol"].iloc[-1] if exit_time != expiry_time else None
        })
        
        # Calculate hourly PnL for the holding period
        hourly_prices = pd.merge(
            call_prices[["timestamp", "price"]].rename(columns={"price": "call_price"}),
            put_prices[["timestamp", "price"]].rename(columns={"price": "put_price"}),
            on="timestamp",
            how="outer"
        )
        hourly_prices = hourly_prices[
            (hourly_prices["timestamp"] >= entry_time) &
            (hourly_prices["timestamp"] <= exit_time)
        ]
        
        hourly_prices["call_price"] = hourly_prices["call_price"].ffill()
        hourly_prices["put_price"] = hourly_prices["put_price"].ffill()
        
        hourly_prices["call_hourly_pnl"] = (hourly_prices["call_price"] - call_entry["price"]) * 1
        hourly_prices["put_hourly_pnl"] = (hourly_prices["put_price"] - put_entry["price"]) * 1
        hourly_prices["total_hourly_pnl"] = hourly_prices["call_hourly_pnl"] + hourly_prices["put_hourly_pnl"]
        hourly_prices["entry_time"] = entry_time
        
        hourly_pnl_records.append(hourly_prices[["timestamp", "entry_time", "call_hourly_pnl", "put_hourly_pnl", "total_hourly_pnl"]])
    
    trade_df = pd.DataFrame(trade_results)
    hourly_pnl_df = pd.concat(hourly_pnl_records, ignore_index=True) if hourly_pnl_records else pd.DataFrame()
    
    if not trade_df.empty:
        trade_df = trade_df.sort_values("entry_time")
    if not hourly_pnl_df.empty:
        hourly_pnl_df = hourly_pnl_df.sort_values(["timestamp", "entry_time"])
    
    return trade_df, hourly_pnl_df

In [83]:
trade_df, hourly_pnl_df = backtest_weekly_straddle_buy_low_dvol(days=840, dvol_buy_threshold=48, dvol_sell_threshold=55)

# Calculate and display summary statistics
stats = calculate_summary_statistics(trade_df)
print("\nTrade Summary:")
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title()}: {value:.2f}" if isinstance(value, float) else f"{key.replace('_', ' ').title()}: {value}")

# Generate and display plots
pnl_curve_fig, pnl_bar_fig = plot_pnl_charts(trade_df)
if pnl_curve_fig:
    pnl_curve_fig.show()
if pnl_bar_fig:
    pnl_bar_fig.show()

Skipping trade at 2023-01-13 00:00:00+00:00: DVOL=51.42 >= 48
Skipping trade at 2023-01-20 00:00:00+00:00: DVOL=56.62 >= 48
Skipping trade at 2023-01-27 00:00:00+00:00: DVOL=56.91 >= 48
Skipping trade at 2023-02-03 00:00:00+00:00: DVOL=57.69 >= 48
Skipping trade at 2023-02-10 00:00:00+00:00: DVOL=50.5 >= 48
Skipping trade at 2023-02-17 00:00:00+00:00: DVOL=58.5 >= 48
Skipping trade at 2023-02-24 00:00:00+00:00: DVOL=54.81 >= 48
Skipping trade at 2023-03-03 00:00:00+00:00: DVOL=53.43 >= 48
Skipping trade at 2023-03-10 00:00:00+00:00: DVOL=54.79 >= 48
Skipping trade at 2023-03-17 00:00:00+00:00: DVOL=61.21 >= 48
Skipping trade at 2023-03-24 00:00:00+00:00: DVOL=66.01 >= 48
Skipping trade at 2023-03-31 00:00:00+00:00: DVOL=62.77 >= 48
Skipping trade at 2023-04-07 00:00:00+00:00: DVOL=56.88 >= 48
Skipping trade at 2023-04-14 00:00:00+00:00: DVOL=55.76 >= 48
Skipping trade at 2023-04-21 00:00:00+00:00: DVOL=55.77 >= 48
Skipping trade at 2023-04-28 00:00:00+00:00: DVOL=54.59 >= 48
Skipping t

In [82]:
fig = plot_hourly_cumulative_pnl(hourly_pnl_df)
fig.show()

In [7]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import time
from spot_vol import fetch_binance_btc_spot_price  # From your provided file

def fetch_dvol_index(trade_time, retries=3):
    """
    Fetch the latest DVOL Index value for BTC before or at the trade time using Deribit API.
    Returns the most recent DVOL value (close price) or None if data is unavailable.
    """
    url = "https://www.deribit.com/api/v2/public/get_volatility_index_data"
    start_ts = int((trade_time - timedelta(hours=24)).timestamp() * 1000)  # 24 hours before
    end_ts = int(trade_time.timestamp() * 1000)
    
    params = {
        "currency": "BTC",
        "start_timestamp": start_ts,
        "end_timestamp": end_ts,
        "resolution": "3600"  # 3600 seconds = 1 hour
    }
    
    for attempt in range(retries):
        try:
            time.sleep(0.1)  # Avoid rate limits
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'result' not in data or 'data' not in data['result']:
                print(f"No DVOL data for BTC from {trade_time - timedelta(hours=24)} to {trade_time}")
                return None
            
            # data['result']['data'] is a list of [timestamp_ms, open, high, low, close]
            dvol_data = data['result']['data']
            if not dvol_data:
                print(f"No DVOL data points available before {trade_time}")
                return None
            
            # Extract timestamps and close values
            df = pd.DataFrame(
                dvol_data,
                columns=["timestamp_ms", "open", "high", "low", "dvol"]
            )
            df["timestamp"] = pd.to_datetime(df["timestamp_ms"], unit="ms", utc=True)
            df = df[df["timestamp"] <= trade_time]
            
            if df.empty:
                print(f"No DVOL data available before {trade_time}")
                return None
            
            # Get the most recent DVOL value
            latest_dvol = df.sort_values("timestamp").iloc[-1]["dvol"]
            return latest_dvol
        
        except requests.RequestException as e:
            print(f"Attempt {attempt+1} failed for DVOL fetch at {trade_time}: {e}")
            if attempt == retries - 1:
                return None
            time.sleep(1)

def backtest_weekly_straddle(days=365, dvol_threshold=55):
    """
    Backtest selling a BTC weekly straddle every Friday at 00:00 UTC if DVOL Index > 55.
    Returns a DataFrame with trade results and a DataFrame with hourly PnL.
    """
    end_dt = datetime.now(pytz.UTC).replace(hour=0, minute=0, second=0, microsecond=0)
    start_dt = end_dt - timedelta(days=days)
    
    btc_df = fetch_binance_btc_spot_price(start_dt, end_dt)
    if btc_df.empty:
        print("No BTC spot price data. Cannot proceed with backtest.")
        return pd.DataFrame(), pd.DataFrame()
    
    fridays = get_friday_dates(start_dt, end_dt)
    
    trade_results = []
    hourly_pnl_records = []
    
    for entry_time in fridays:
        # Check DVOL Index value
        dvol_value = fetch_dvol_index(entry_time)
        if dvol_value is None or dvol_value <= dvol_threshold:
            print(f"Skipping trade at {entry_time}: DVOL={dvol_value if dvol_value else 'N/A'} <= {dvol_threshold}")
            continue
        
        entry_date = entry_time.date()
        spot_price_row = btc_df[btc_df["date"] == entry_date]
        
        if spot_price_row.empty:
            print(f"No spot price data for {entry_date}. Skipping trade.")
            continue
        
        spot_price = spot_price_row["price"].iloc[0]
        
        call_strike = round_strike_price(spot_price, is_call=True)
        put_strike = round_strike_price(spot_price, is_call=False)
        
        expiry_time = entry_time + timedelta(days=7)
        expiry_time = expiry_time.replace(hour=8)
        
        call_ticker = generate_deribit_ticker("BTC", call_strike, expiry_time, "C")
        put_ticker = generate_deribit_ticker("BTC", put_strike, expiry_time, "P")
        
        if not (check_data_availability(call_ticker, entry_time, expiry_time) and 
                check_data_availability(put_ticker, entry_time, expiry_time)):
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_prices = fetch_deribit_option_prices(call_ticker, entry_time, expiry_time)
        put_prices = fetch_deribit_option_prices(put_ticker, entry_time, expiry_time)
        
        if call_prices.empty or put_prices.empty:
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_entry = call_prices[call_prices["timestamp"] >= entry_time].iloc[0]
        put_entry = put_prices[put_prices["timestamp"] >= entry_time].iloc[0]
        
        call_exit = call_prices[call_prices["timestamp"] <= expiry_time].iloc[-1]
        put_exit = put_prices[put_prices["timestamp"] <= expiry_time].iloc[-1]
        
        call_pnl = (call_entry["price"] - call_exit["price"]) * 1
        put_pnl = (put_entry["price"] - put_exit["price"]) * 1
        total_pnl = call_pnl + put_pnl
        
        trade_results.append({
            "entry_time": entry_time,
            "call_ticker": call_ticker,
            "call_strike": call_strike,
            "put_ticker": put_ticker,
            "put_strike": put_strike,
            "expiry_time": expiry_time,
            "call_entry_price": call_entry["price"],
            "put_entry_price": put_entry["price"],
            "call_exit_price": call_exit["price"],
            "put_exit_price": put_exit["price"],
            "call_pnl": call_pnl,
            "put_pnl": put_pnl,
            "total_pnl": total_pnl,
            "dvol_value": dvol_value
        })
        
        hourly_prices = pd.merge(
            call_prices[["timestamp", "price"]].rename(columns={"price": "call_price"}),
            put_prices[["timestamp", "price"]].rename(columns={"price": "put_price"}),
            on="timestamp",
            how="outer"
        )
        hourly_prices = hourly_prices[
            (hourly_prices["timestamp"] >= entry_time) &
            (hourly_prices["timestamp"] <= expiry_time)
        ]
        
        hourly_prices["call_price"] = hourly_prices["call_price"].ffill()
        hourly_prices["put_price"] = hourly_prices["put_price"].ffill()
        
        hourly_prices["call_hourly_pnl"] = (call_entry["price"] - hourly_prices["call_price"]) * 1
        hourly_prices["put_hourly_pnl"] = (put_entry["price"] - hourly_prices["put_price"]) * 1
        hourly_prices["total_hourly_pnl"] = hourly_prices["call_hourly_pnl"] + hourly_prices["put_hourly_pnl"]
        hourly_prices["entry_time"] = entry_time
        
        hourly_pnl_records.append(hourly_prices[["timestamp", "entry_time", "call_hourly_pnl", "put_hourly_pnl", "total_hourly_pnl"]])
    
    trade_df = pd.DataFrame(trade_results)
    hourly_pnl_df = pd.concat(hourly_pnl_records, ignore_index=True) if hourly_pnl_records else pd.DataFrame()
    
    if not trade_df.empty:
        trade_df = trade_df.sort_values("entry_time")
    if not hourly_pnl_df.empty:
        hourly_pnl_df = hourly_pnl_df.sort_values(["timestamp", "entry_time"])
    
    return trade_df, hourly_pnl_df

In [ ]:
trade_df, hourly_pnl_df = backtest_weekly_straddle(days=1000, dvol_threshold=55)

Skipping trade at 2022-11-04 00:00:00+00:00: DVOL=53.74 <= 55
Skipping trade at 2022-12-30 00:00:00+00:00: DVOL=54.65 <= 55
Skipping trade at 2023-01-06 00:00:00+00:00: DVOL=45.38 <= 55
Skipping trade at 2023-01-13 00:00:00+00:00: DVOL=51.42 <= 55
Skipping trade at 2023-02-10 00:00:00+00:00: DVOL=50.5 <= 55
Skipping trade at 2023-02-24 00:00:00+00:00: DVOL=54.81 <= 55
Skipping trade at 2023-03-03 00:00:00+00:00: DVOL=53.43 <= 55
Skipping trade at 2023-03-10 00:00:00+00:00: DVOL=54.79 <= 55
Skipping trade at 2023-04-28 00:00:00+00:00: DVOL=54.59 <= 55
Skipping trade at 2023-05-05 00:00:00+00:00: DVOL=51.91 <= 55
Skipping trade at 2023-05-12 00:00:00+00:00: DVOL=51.02 <= 55
Skipping trade at 2023-05-19 00:00:00+00:00: DVOL=46.21 <= 55
Skipping trade at 2023-05-26 00:00:00+00:00: DVOL=47.63 <= 55
Skipping trade at 2023-06-02 00:00:00+00:00: DVOL=42.15 <= 55
Skipping trade at 2023-06-09 00:00:00+00:00: DVOL=41.28 <= 55
Skipping trade at 2023-06-16 00:00:00+00:00: DVOL=46.42 <= 55
Skipping 

In [22]:
# Generate and display plots
pnl_curve_fig, pnl_bar_fig = plot_pnl_charts(trade_df)
if pnl_curve_fig:
    pnl_curve_fig.show()
if pnl_bar_fig:
    pnl_bar_fig.show()

In [23]:
fig = plot_hourly_cumulative_pnl(hourly_pnl_df)
fig.show()

In [24]:
# Calculate and display summary statistics
stats = calculate_summary_statistics(trade_df)
print("\nTrade Summary:")
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title()}: {value:.2f}" if isinstance(value, float) else f"{key.replace('_', ' ').title()}: {value}")


Trade Summary:
Number Of Trades: 71
Total Pnl: 1.35
Average Pnl Per Trade: 0.02
Win Rate: 77.46
Max Drawdown: 0.13


# strangles

In [27]:
def backtest_weekly_strangle(days=365, dvol_threshold=55):
    """
    Backtest selling a BTC weekly strangle (105% call, 95% put) every Friday at 00:00 UTC if DVOL Index > 55.
    Returns a DataFrame with trade results and a DataFrame with hourly PnL.
    """
    end_dt = datetime.now(pytz.UTC).replace(hour=0, minute=0, second=0, microsecond=0)
    start_dt = end_dt - timedelta(days=days)
    
    btc_df = fetch_binance_btc_spot_price(start_dt, end_dt)
    if btc_df.empty:
        print("No BTC spot price data. Cannot proceed with backtest.")
        return pd.DataFrame(), pd.DataFrame()
    
    fridays = get_friday_dates(start_dt, end_dt)
    
    trade_results = []
    hourly_pnl_records = []
    
    for entry_time in fridays:
        dvol_value = fetch_dvol_index(entry_time)
        if dvol_value is None or dvol_value <= dvol_threshold:
            print(f"Skipping trade at {entry_time}: DVOL={dvol_value if dvol_value else 'N/A'} <= {dvol_threshold}")
            continue
        
        entry_date = entry_time.date()
        spot_price_row = btc_df[btc_df["date"] == entry_date]
        
        if spot_price_row.empty:
            print(f"No spot price data for {entry_date}. Skipping trade.")
            continue
        
        spot_price = spot_price_row["price"].iloc[0]
        
        # Calculate strangle strikes: 110% for call, 90% for put
        call_strike = round_strike_price(spot_price * 1.05, is_call=True)
        put_strike = round_strike_price(spot_price * 0.95, is_call=False)
        
        expiry_time = entry_time + timedelta(days=7)
        expiry_time = expiry_time.replace(hour=8)
        
        call_ticker = generate_deribit_ticker("BTC", call_strike, expiry_time, "C")
        put_ticker = generate_deribit_ticker("BTC", put_strike, expiry_time, "P")
        
        if not (check_data_availability(call_ticker, entry_time, expiry_time) and 
                check_data_availability(put_ticker, entry_time, expiry_time)):
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_prices = fetch_deribit_option_prices(call_ticker, entry_time, expiry_time)
        put_prices = fetch_deribit_option_prices(put_ticker, entry_time, expiry_time)
        
        if call_prices.empty or put_prices.empty:
            print(f"No price data for {call_ticker} or {put_ticker}. Skipping trade.")
            continue
        
        call_entry = call_prices[call_prices["timestamp"] >= entry_time].iloc[0]
        put_entry = put_prices[put_prices["timestamp"] >= entry_time].iloc[0]
        
        call_exit = call_prices[call_prices["timestamp"] <= expiry_time].iloc[-1]
        put_exit = put_prices[put_prices["timestamp"] <= expiry_time].iloc[-1]
        
        call_pnl = (call_entry["price"] - call_exit["price"]) * 1
        put_pnl = (put_entry["price"] - put_exit["price"]) * 1
        total_pnl = call_pnl + put_pnl
        
        trade_results.append({
            "entry_time": entry_time,
            "call_ticker": call_ticker,
            "call_strike": call_strike,
            "put_ticker": put_ticker,
            "put_strike": put_strike,
            "expiry_time": expiry_time,
            "call_entry_price": call_entry["price"],
            "put_entry_price": put_entry["price"],
            "call_exit_price": call_exit["price"],
            "put_exit_price": put_exit["price"],
            "call_pnl": call_pnl,
            "put_pnl": put_pnl,
            "total_pnl": total_pnl,
            "dvol_value": dvol_value
        })
        
        hourly_prices = pd.merge(
            call_prices[["timestamp", "price"]].rename(columns={"price": "call_price"}),
            put_prices[["timestamp", "price"]].rename(columns={"price": "put_price"}),
            on="timestamp",
            how="outer"
        )
        hourly_prices = hourly_prices[
            (hourly_prices["timestamp"] >= entry_time) &
            (hourly_prices["timestamp"] <= expiry_time)
        ]
        
        hourly_prices["call_price"] = hourly_prices["call_price"].ffill()
        hourly_prices["put_price"] = hourly_prices["put_price"].ffill()
        
        hourly_prices["call_hourly_pnl"] = (call_entry["price"] - hourly_prices["call_price"]) * 1
        hourly_prices["put_hourly_pnl"] = (put_entry["price"] - hourly_prices["put_price"]) * 1
        hourly_prices["total_hourly_pnl"] = hourly_prices["call_hourly_pnl"] + hourly_prices["put_hourly_pnl"]
        hourly_prices["entry_time"] = entry_time
        
        hourly_pnl_records.append(hourly_prices[["timestamp", "entry_time", "call_hourly_pnl", "put_hourly_pnl", "total_hourly_pnl"]])
    
    trade_df = pd.DataFrame(trade_results)
    hourly_pnl_df = pd.concat(hourly_pnl_records, ignore_index=True) if hourly_pnl_records else pd.DataFrame()
    
    if not trade_df.empty:
        trade_df = trade_df.sort_values("entry_time")
    if not hourly_pnl_df.empty:
        hourly_pnl_df = hourly_pnl_df.sort_values(["timestamp", "entry_time"])
    
    return trade_df, hourly_pnl_df

In [30]:
trade_df, hourly_pnl_df = backtest_weekly_strangle(days=1000, dvol_threshold=55)

Skipping trade at 2022-11-04 00:00:00+00:00: DVOL=53.74 <= 55
Skipping trade at 2022-12-30 00:00:00+00:00: DVOL=54.65 <= 55
Skipping trade at 2023-01-06 00:00:00+00:00: DVOL=45.38 <= 55
Skipping trade at 2023-01-13 00:00:00+00:00: DVOL=51.42 <= 55
Skipping trade at 2023-02-10 00:00:00+00:00: DVOL=50.5 <= 55
Skipping trade at 2023-02-24 00:00:00+00:00: DVOL=54.81 <= 55
Skipping trade at 2023-03-03 00:00:00+00:00: DVOL=53.43 <= 55
Skipping trade at 2023-03-10 00:00:00+00:00: DVOL=54.79 <= 55
Skipping trade at 2023-04-28 00:00:00+00:00: DVOL=54.59 <= 55
Skipping trade at 2023-05-05 00:00:00+00:00: DVOL=51.91 <= 55
Skipping trade at 2023-05-12 00:00:00+00:00: DVOL=51.02 <= 55
Skipping trade at 2023-05-19 00:00:00+00:00: DVOL=46.21 <= 55
Skipping trade at 2023-05-26 00:00:00+00:00: DVOL=47.63 <= 55
Skipping trade at 2023-06-02 00:00:00+00:00: DVOL=42.15 <= 55
Skipping trade at 2023-06-09 00:00:00+00:00: DVOL=41.28 <= 55
Skipping trade at 2023-06-16 00:00:00+00:00: DVOL=46.42 <= 55
Skipping 

In [31]:
# Calculate and display summary statistics
stats = calculate_summary_statistics(trade_df)
print("\nTrade Summary:")
for key, value in stats.items():
    print(f"{key.replace('_', ' ').title()}: {value:.2f}" if isinstance(value, float) else f"{key.replace('_', ' ').title()}: {value}")
# Generate and display plots
pnl_curve_fig, pnl_bar_fig = plot_pnl_charts(trade_df)
if pnl_curve_fig:
    pnl_curve_fig.show()
if pnl_bar_fig:
    pnl_bar_fig.show()


Trade Summary:
Number Of Trades: 71
Total Pnl: 0.92
Average Pnl Per Trade: 0.01
Win Rate: 85.92
Max Drawdown: 0.11


In [33]:
import requests
import pandas as pd
import plotly.express as px
from datetime import datetime

# Function to fetch options data from Deribit API
def get_deribit_options_data(currency):
    url = "https://www.deribit.com/api/v2/public/get_book_summary_by_currency"
    params = {"currency": currency, "kind": "option"}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if data.get("result"):
            return data["result"]
        else:
            print(f"No data returned for {currency}")
            return []
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return []

# Function to process data and extract relevant fields
def process_options_data(options_data):
    records = []
    for item in options_data:
        instrument_name = item.get("instrument_name", "")
        # Example instrument_name: BTC-31DEC25-100000-C
        parts = instrument_name.split("-")
        if len(parts) != 4:
            continue
        coin, expiry, strike, option_type = parts
        expiry_date = datetime.strptime(expiry, "%d%b%y")
        records.append({
            "strike_price": float(strike),
            "expiry_date": expiry_date,
            "option_type": option_type,  # C for Call, P for Put
            "open_interest": item.get("open_interest", 0)
        })
    return pd.DataFrame(records)

# Main function to create and display the bubble chart
def create_bubble_chart(currency = "BTC"):

    # Fetch and process data
    options_data = get_deribit_options_data(currency)
    if not options_data:
        print("No data to display.")
        return

    df = process_options_data(options_data)

    if df.empty:
        print("No valid options data to display.")
        return

    # Create bubble chart
    fig = px.scatter(
        df,
        x="strike_price",
        y="expiry_date",
        size="open_interest",
        color="option_type",
        hover_data=["open_interest", "option_type"],
        title=f"{currency} Options Open Interest on Deribit",
        labels={
            "strike_price": "Strike Price",
            "expiry_date": "Expiration Date",
            "option_type": "Option Type"
        },
        color_discrete_map={"C": "blue", "P": "red"}
    )

    # Update layout for better visualization
    fig.update_traces(marker=dict(sizemode="area", sizeref=2.*max(df["open_interest"])/(100.**2), line=dict(width=0)))
    fig.update_layout(
        xaxis_title="Strike Price",
        yaxis_title="Expiration Date",
        showlegend=True,
        height=600
    )

    fig.show()



In [34]:
create_bubble_chart(currency="BTC")

In [35]:
create_bubble_chart(currency="ETH")

In [ ]:
import requests
import pandas as pd
import plotly.express as px
from datetime import datetime

# Function to fetch options data from Deribit API
def get_deribit_options_data(currency):
    url = "https://www.deribit.com/api/v2/public/get_book_summary_by_currency"
    params = {"currency": currency, "kind": "option"}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if data.get("result"):
            return data["result"]
        else:
            print(f"No data returned for {currency}")
            return []
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return []

# Function to process data and extract relevant fields
def process_options_data(options_data):
    records = []
    for item in options_data:
        instrument_name = item.get("instrument_name", "")
        # Example instrument_name: BTC-31DEC25-100000-C
        parts = instrument_name.split("-")
        if len(parts) != 4:
            continue
        coin, expiry, strike, option_type = parts
        expiry_date = datetime.strptime(expiry, "%d%b%y")
        records.append({
            "strike_price": float(strike),
            "expiry_date": expiry_date,
            "option_type": option_type,  # C for Call, P for Put
            "open_interest": item.get("open_interest", 0)
        })
    return pd.DataFrame(records)

# Main function to create and display the bubble chart
def create_bubble_chart(currency='BTC'):
    # Fetch and process data
    options_data = get_deribit_options_data(currency)
    if not options_data:
        print("No data to display.")
        return

    df = process_options_data(options_data)

    if df.empty:
        print("No valid options data to display.")
        return

    # Create bubble chart
    fig = px.scatter(
        df,
        x="strike_price",
        y="expiry_date",
        size="open_interest",
        color="option_type",
        hover_data=["open_interest", "option_type"],
        title=f"{currency} Options Open Interest on Deribit",
        labels={
            "strike_price": "Strike Price",
            "expiry_date": "Expiration Date",
            "option_type": "Option Type"
        },
        color_discrete_map={"C": "#009392", "P": "#d77d8f"}  # Softer blue and orange
    )

    # Update traces for bubble appearance
    max_oi = max(df["open_interest"], default=1)
    fig.update_traces(
        marker=dict(
            sizemode="area",
            sizeref=2. * max_oi / (80.**2),  # Adjusted for better scaling
            line=dict(width=0.5, color="DarkSlateGrey"),
            opacity=0.7
        ),
        hovertemplate=(
            "<b>%{customdata[1]}</b><br>" +
            "Strike: $%{x:,.0f}<br>" +
            "Expiry: %{y|%b %d, %Y}<br>" +
            "Open Interest: %{customdata[0]:,.2f}<extra></extra>"
        )
    )

    # Update layout for aesthetics and readability
    fig.update_layout(
        title=dict(
            x=0.5,
            xanchor="center",
            font=dict(size=20, family="Arial, sans-serif")
        ),
        xaxis=dict(
            title="Strike Price",
            tickformat=",.0f",
            gridcolor="LightGrey",
            zeroline=False
        ),
        yaxis=dict(
            title="Expiration Date",
            tickformat="%b %d, %Y",
            gridcolor="LightGrey",
            zeroline=False
        ),
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=True,
        legend=dict(
            title="Option Type",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        ),
        height=700,  # Slightly larger chart
        margin=dict(l=50, r=50, t=100, b=50),
        font=dict(family="Arial, sans-serif", size=12)
    )

    # Add annotation for bubble size
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=0.01,
        y=0.01,
        text="Bubble size represents open interest",
        showarrow=False,
        font=dict(size=10, color="grey"),
        align="left"
    )

    fig.show()

create_bubble_chart(currency='ETH')

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
import requests

def clean_ohlc_data(df):
    """
    Cleans OHLC data by dropping rows where any OHLC value is over 2x or under 0.5x
    the median of adjacent points (previous and next hour).
    """
    df_clean = df.copy()
    columns_to_check = ["open", "high", "low", "close"]
    
    # Initialize a mask for rows to keep (True means keep, False means drop)
    keep_rows = pd.Series(True, index=df_clean.index)
    
    for col in columns_to_check:
        # Calculate median of previous and next hour for each point
        prev_values = df_clean[col].shift(1)
        next_values = df_clean[col].shift(-1)
        median_adjacent = pd.concat([prev_values, next_values], axis=1).median(axis=1)
        
        # Identify outliers: >2x or <0.5x the median of adjacent points
        ratio = df_clean[col] / median_adjacent
        outliers = (ratio > 2) | (ratio < 0.5)
        
        # Update mask: mark rows with outliers to be dropped
        keep_rows = keep_rows & ~outliers
    
    # Drop rows where any column has an outlier
    df_clean = df_clean[keep_rows]
    
    return df_clean

def fetch_binance_btc_spot_price(start_date, end_date):
    """
    Fetch hourly BTC spot price (OHLC) from Binance API for a given date range, handling pagination.
    Returns a DataFrame with hourly OHLC data.
    """
    url = "https://api.binance.us/api/v3/klines"
    chunk_days = 30  # Each chunk covers 30 days (~720 hours, within limit=1000)
    all_data = []
    
    current_start = start_date
    while current_start < end_date:
        current_end = min(current_start + timedelta(days=chunk_days), end_date)
        start_ts = int(current_start.timestamp() * 1000)
        end_ts = int(current_end.timestamp() * 1000)
        
        params = {
            "symbol": "BTCUSDT",
            "interval": "1h",
            "startTime": start_ts,
            "endTime": end_ts,
            "limit": 1000
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if not data:
                print(f"No Binance data for {current_start.date()} to {current_end.date()}")
                current_start = current_end
                continue
            
            df = pd.DataFrame(
                data,
                columns=[
                    "timestamp", "open", "high", "low", "close", "volume",
                    "close_time", "quote_asset_volume", "number_of_trades",
                    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
                ]
            )
            
            df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
            df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
            
            # Clean the OHLC data
            df = clean_ohlc_data(df)
            
            all_data.append(df)
            
        except requests.RequestException as e:
            print(f"Error fetching Binance BTC price data for {current_start.date()} to {current_end.date()}: {e}")
        
        current_start = current_end
    
    if not all_data:
        print("No Binance data retrieved")
        return pd.DataFrame()
    
    # Consolidate data
    df = pd.concat(all_data, ignore_index=True)
    df = df.drop_duplicates(subset=["timestamp"], keep="first")
    df = df.sort_values("timestamp")
    
    # Reset index to ensure continuity
    df = df.reset_index(drop=True)
    
    return df[["timestamp", "open", "high", "low", "close"]]

def calculate_statistics(pnl_df, initial_capital=1000):
    """
    Calculate performance statistics: Sharpe Ratio, Max Drawdown, Annualized Return,
    Total Trades, Win Rate, Average Trade PnL.
    """
    # Calculate hourly returns
    portfolio_values = pnl_df['pnl'] + initial_capital
    returns = portfolio_values.pct_change().dropna()
    
    # Sharpe Ratio (annualized, assuming risk-free rate = 0)
    mean_return = returns.mean()
    std_return = returns.std()
    hours_per_year = 365 * 24
    sharpe_ratio = (mean_return / std_return) * np.sqrt(hours_per_year) if std_return != 0 else 0
    
    # Maximum Drawdown
    cumulative_max = portfolio_values.cummax()
    drawdowns = (cumulative_max - portfolio_values) / cumulative_max
    max_drawdown = drawdowns.max() * 100  # As percentage
    
    # Annualized Return
    total_return = (portfolio_values.iloc[-1] / portfolio_values.iloc[0]) - 1
    days = (pnl_df['timestamp'].iloc[-1] - pnl_df['timestamp'].iloc[0]).total_seconds() / (24 * 3600)
    annualized_return = ((1 + total_return) ** (365 / days) - 1) * 100  # As percentage
    
    # Trade statistics
    trades = pnl_df[pnl_df['trade_pnl'].notna()]
    total_trades = len(trades)
    win_trades = len(trades[trades['trade_pnl'] > 0])
    win_rate = (win_trades / total_trades * 100) if total_trades > 0 else 0
    avg_trade_pnl = trades['trade_pnl'].mean() if total_trades > 0 else 0
    
    return {
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown (%)': max_drawdown,
        'Annualized Return (%)': annualized_return,
        'Total Trades': total_trades,
        'Win Rate (%)': win_rate,
        'Average Trade PnL (USD)': avg_trade_pnl
    }

# Define date range
start_date = datetime(2024, 1, 1, tzinfo=timezone.utc)
end_date = datetime(2025, 5, 14, tzinfo=timezone.utc)

# Fetch hourly BTC data
df = fetch_binance_btc_spot_price(start_date, end_date)

if df.empty:
    print("No data fetched. Exiting.")
    exit()

# Ensure data is sufficient for MA calculations
if len(df) < 10:
    print("Insufficient data for 10-hour MA. Exiting.")
    exit()

# Calculate moving averages
df['fast_ma'] = df['close'].rolling(window=5).mean()
df['slow_ma'] = df['close'].rolling(window=10).mean()

# Generate signals
df['signal'] = 0
df['fast_ma_prev'] = df['fast_ma'].shift(1)
df['slow_ma_prev'] = df['slow_ma'].shift(1)

# Long signal: fast MA crosses above slow MA
df.loc[(df['fast_ma'] > df['slow_ma']) & (df['fast_ma_prev'] <= df['slow_ma_prev']), 'signal'] = 1
# Short signal: fast MA crosses below slow MA
df.loc[(df['fast_ma'] < df['slow_ma']) & (df['fast_ma_prev'] >= df['slow_ma_prev']), 'signal'] = -1

# Backtest parameters
investment = 1000  # USD per position
initial_capital = 1000  # Starting capital for return calculations
position = 0  # 1 for long, -1 for short, 0 for flat
btc_held = 0  # BTC units held
cash = 0  # USD cash balance
pnl = []  # Store PnL over time
pnl_timestamps = []  # Store corresponding timestamps
trade_pnl = []  # Store per-trade PnL

# Backtest loop
for i in range(1, len(df)):
    if pd.isna(df['close'].iloc[i]) or pd.isna(df['signal'].iloc[i]):
        continue  # Skip rows with missing data
    price = df['close'].iloc[i]
    signal = df['signal'].iloc[i]
    
    # Initialize trade profit as NaN for this step
    current_trade_pnl = np.nan
    
    # Execute trades based on signal
    if signal == 1 and position != 1:  # Enter long
        if position == -1:  # Close short
            current_trade_pnl = btc_held * price + investment  # Realized PnL
            cash += btc_held * price
            btc_held = 0
        btc_held += investment / price  # Buy BTC with $1000
        cash -= investment
        position = 1
    elif signal == -1 and position != -1:  # Enter short
        if position == 1:  # Close long
            current_trade_pnl = btc_held * price - investment  # Realized PnL
            cash += btc_held * price
            btc_held = 0
        btc_held -= investment / price  # Short BTC worth $1000
        cash += investment
        position = -1
    
    # Calculate PnL
    portfolio_value = cash + btc_held * price
    pnl.append(portfolio_value)
    pnl_timestamps.append(df['timestamp'].iloc[i])
    trade_pnl.append(current_trade_pnl)

# Check if PnL data exists
if not pnl:
    print("No valid trades executed. Exiting.")
    exit()

# Debug: Verify list lengths
print(f"Length of pnl: {len(pnl)}")
print(f"Length of pnl_timestamps: {len(pnl_timestamps)}")
print(f"Length of trade_pnl: {len(trade_pnl)}")

# Ensure all lists have the same length
min_length = min(len(pnl), len(pnl_timestamps), len(trade_pnl))
pnl = pnl[:min_length]
pnl_timestamps = pnl_timestamps[:min_length]
trade_pnl = trade_pnl[:min_length]

# Create PnL DataFrame for plotting and analysis
pnl_df = pd.DataFrame({'timestamp': pnl_timestamps, 'pnl': pnl, 'trade_pnl': trade_pnl})

# Calculate statistics
stats = calculate_statistics(pnl_df, initial_capital)

# Print statistics
print("\nPerformance Statistics:")
for key, value in stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

# Plot PnL
plt.figure(figsize=(10, 6))
plt.plot(pnl_df['timestamp'], pnl_df['pnl'], label='Cumulative PnL (USD)')
plt.title('Trend-Following Strategy PnL')
plt.xlabel('Time')
plt.ylabel('PnL (USD)')
plt.grid(True)
plt.legend()
plt.savefig('pnl_chart.png')
plt.close()

# Print final PnL
final_pnl = pnl_df['pnl'].iloc[-1]
print(f"\nFinal PnL: ${final_pnl:.2f}")

Length of pnl: 11976
Length of pnl_timestamps: 11976
Length of trade_pnl: 11976

Performance Statistics:
Sharpe Ratio: 1.36
Max Drawdown (%): 20.25
Annualized Return (%): 56.59
Total Trades: 1336
Win Rate (%): 38.02
Average Trade PnL (USD): 0.62

Final PnL: $845.99


In [49]:
import pandas as pd
import plotly.graph_objects as go
from binance.client import Client
from datetime import datetime
import numpy as np

# Initialize Binance client (no API key needed for public data)
# Note: If rate limits are hit, register for an API key at binance.com
client = Client()

# Set variables for date range and interval
start_date = '2025-01-01'  # Start date in YYYY-MM-DD format
end_date = '2025-05-28'    # End date in YYYY-MM-DD format
return_interval = 3         # Number of days for return calculation (e.g., 2 for 2-day returns)

# Function to fetch historical daily data
def get_historical_data(symbol, interval, start_date, end_date):
    klines = client.get_historical_klines(symbol, interval, start_date, end_date)
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'num_trades',
        'taker_buy_base_volume', 'taker_buy_quote_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['close'] = df['close'].astype(float)
    return df[['timestamp', 'close']]

# Function to calculate n-day returns
def calculate_interval_returns(df, interval):
    # Keep every nth row for the specified interval
    df_interval = df.iloc[::interval].copy()
    # Calculate returns based on close prices
    df_interval['returns'] = df_interval['close'].pct_change() * 100  # Returns in percentage
    return df_interval

# Fetch Bitcoin daily data (BTCUSDT)
df = get_historical_data('BTCUSDT', Client.KLINE_INTERVAL_1DAY, start_date, end_date)

# Calculate returns for the specified interval
df_returns = calculate_interval_returns(df, return_interval)

# Calculate standard deviations
mean_return = df_returns['returns'].mean()
std_return = df_returns['returns'].std()
sd1_upper = mean_return + std_return
sd1_lower = mean_return - std_return
sd2_upper = mean_return + 2 * std_return
sd2_lower = mean_return - 2 * std_return

# Create Plotly figure
fig = go.Figure()

# Add bar chart for interval returns
fig.add_trace(go.Bar(
    x=df_returns['timestamp'],
    y=df_returns['returns'],
    name=f'{return_interval}-Day Returns (%)',
    marker_color=['green' if x >= 0 else 'red' for x in df_returns['returns']]
))

# Add standard deviation lines
fig.add_trace(go.Scatter(
    x=df_returns['timestamp'], y=[sd1_upper] * len(df_returns),
    name='+1 SD', line=dict(color='blue', dash='dash')
))
fig.add_trace(go.Scatter(
    x=df_returns['timestamp'], y=[sd1_lower] * len(df_returns),
    name='-1 SD', line=dict(color='blue', dash='dash')
))
fig.add_trace(go.Scatter(
    x=df_returns['timestamp'], y=[sd2_upper] * len(df_returns),
    name='+2 SD', line=dict(color='purple', dash='dot')
))
fig.add_trace(go.Scatter(
    x=df_returns['timestamp'], y=[sd2_lower] * len(df_returns),
    name='-2 SD', line=dict(color='purple', dash='dot')
))

# Update layout
fig.update_layout(
    title=f'Bitcoin {return_interval}-Day Returns with Standard Deviation Lines',
    xaxis_title='Date',
    yaxis_title=f'{return_interval}-Day Return (%)',
    template='plotly_white',
    showlegend=True
)

# Show plot
fig.show()

In [67]:
def historical_return_histogram(start_date, end_date, return_interval = 1):
    # Initialize Binance client (no API key needed for public data)
    # Note: If rate limits are hit, register for an API key at binance.com
    client = Client()
   

    # Function to fetch historical daily data
    def get_historical_data(symbol, interval, start_date, end_date):
        klines = client.get_historical_klines(symbol, interval, start_date, end_date)
        df = pd.DataFrame(klines, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'num_trades',
            'taker_buy_base_volume', 'taker_buy_quote_volume', 'ignore'
        ])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df['close'] = df['close'].astype(float)
        return df[['timestamp', 'close']]

    # Function to calculate n-day returns
    def calculate_interval_returns(df, interval):
        # Keep every nth row for the specified interval
        df_interval = df.iloc[::interval].copy()
        # Calculate returns based on close prices
        df_interval['returns'] = df_interval['close'].pct_change() * 100  # Returns in percentage
        return df_interval

    # Fetch Bitcoin daily data (BTCUSDT)
    df = get_historical_data('BTCUSDT', Client.KLINE_INTERVAL_1DAY, start_date, end_date)

    # Calculate returns for the specified interval
    df_returns = calculate_interval_returns(df, return_interval)

    # Calculate standard deviations
    mean_return = df_returns['returns'].mean()
    std_return = df_returns['returns'].std()
    sd1_upper = mean_return + std_return
    sd1_lower = mean_return - std_return
    sd2_upper = mean_return + 2 * std_return
    sd2_lower = mean_return - 2 * std_return

    # Create Plotly figure
    fig = go.Figure()

    # Add bar chart for interval returns
    fig.add_trace(go.Bar(
        x=df_returns['timestamp'],
        y=df_returns['returns'],
        name=f'{return_interval}-Day Returns (%)',
        marker_color=['rgb(0, 147, 146)' if x >= 0 else 'rgb(208, 88, 126)' for x in df_returns['returns']]
    ))

    # Add standard deviation lines
    fig.add_trace(go.Scatter(
        x=df_returns['timestamp'], y=[sd1_upper] * len(df_returns),
        name='+1 SD', line=dict(color='rgb(0, 223, 221)', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=df_returns['timestamp'], y=[sd1_lower] * len(df_returns),
        name='-1 SD', line=dict(color='rgb(0, 223, 221)', dash='dash')
    ))
    fig.add_trace(go.Scatter(
        x=df_returns['timestamp'], y=[sd2_upper] * len(df_returns),
        name='+2 SD', line=dict(color='rgb(255, 107, 154)', dash='dot')
    ))
    fig.add_trace(go.Scatter(
        x=df_returns['timestamp'], y=[sd2_lower] * len(df_returns),
        name='-2 SD', line=dict(color='rgb(255, 107, 154)', dash='dot')
    ))

    # Update layout
    fig.update_layout(
        title=f'Bitcoin {return_interval}-Day Returns',
        xaxis_title='Date',
        yaxis_title=f'{return_interval}-Day Return (%)',
        template='plotly_white',
        showlegend=True,
        width = 1400,
        height = 600
    )

    # Create Plotly figure for histogram
    fig_2 = go.Figure()

    # Add histogram of returns
    fig_2.add_trace(go.Histogram(
        x=df_returns['returns'].dropna(),  # Drop NaN values from returns
        name=f'{return_interval}-Day Returns',
        nbinsx=30,  # Number of bins for histogram
        opacity=0.7,
        marker_color='skyblue'
    ))

    # Add vertical lines for standard deviations
    fig_2.add_vline(x=sd1_upper, line_dash="dash", line_color="rgb(0, 223, 221)", annotation_text="+1SD", annotation_position="top")
    fig_2.add_vline(x=sd1_lower, line_dash="dash", line_color="rgb(0, 223, 221)", annotation_text="-1SD", annotation_position="top")
    fig_2.add_vline(x=sd2_upper, line_dash="dot", line_color="rgb(255, 107, 154)", annotation_text="+2SD", annotation_position="top")
    fig_2.add_vline(x=sd2_lower, line_dash="dot", line_color="rgb(255, 107, 154)", annotation_text="-2SD", annotation_position="top")

    # Update layout
    fig_2.update_layout(
        title=f'Histogram of Bitcoin {return_interval}-Day Returns ({start_date} to {end_date})',
        xaxis_title=f'{return_interval}-Day Return (%)',
        yaxis_title='Frequency',
        template='plotly_white',
        showlegend=True,
        bargap=0.1,
        width = 1400,
        height = 600
    )

    return fig, fig_2

In [74]:

# Set variables for date range and interval
start_date = '2025-01-01'  # Start date in YYYY-MM-DD format
end_date = '2025-05-28'    # End date in YYYY-MM-DD format
return_interval = 2

fig, fig_2 = historical_return_histogram(start_date=start_date, end_date=end_date, return_interval=return_interval)

In [75]:
fig.show()

In [76]:
fig_2.show()